# [Q.6]

![Alt text](image.png)

## Solution [6]

![Alt text](image-1.png)

In [1]:

# latency is given
latency_X, latency_Y, latency_Z = 30, 40, 20

# response time is given
response_time_A, response_time_B, response_time_C = 60, 40, 80

# calculate AAI for app A on all servers

# App A requires 2-cpu, 8-memory
# resource_utilization = required_resource / available_resource   #<---- sum over all types of resources (memory and cpu)
resource_util_A_X = 2/4 + 4/8 
resource_util_A_Y = 2/4 + 4/8 
resource_util_A_Z = 2/4 + 4/8 
AAI_A_X = 0.4*resource_util_A_X + 0.3*latency_X + 0.3*response_time_A
AAI_A_Y = 0.4*resource_util_A_Y + 0.3*latency_Y + 0.3*response_time_A
AAI_A_Z = 0.4*resource_util_A_Z + 0.3*latency_Z + 0.3*response_time_A

# calculate AAI for app B on all servers
# App B requires 1-cpu, 2-memory
# resource_utilization = required_resource / available_resource   #<---- sum over all types of resources (memory and cpu)
resource_util_B_X = 1/4 + 2/8 
resource_util_B_Y = 1/4 + 2/8 
resource_util_B_Z = 1/4 + 2/8 
AAI_B_X = 0.4*resource_util_B_X + 0.3*latency_X + 0.3*response_time_B
AAI_B_Y = 0.4*resource_util_B_Y + 0.3*latency_Y + 0.3*response_time_B
AAI_B_Z = 0.4*resource_util_B_Z + 0.3*latency_Z + 0.3*response_time_B

# calculate AAI for app C on all servers
# App C requires 3-cpu, 6-memory
# resource_utilization = required_resource / available_resource   #<---- sum over all types of resources (memory and cpu)
resource_util_C_X = 3/4 + 6/8 
resource_util_C_Y = 3/4 + 6/8 
resource_util_C_Z = 3/4 + 6/8 
AAI_C_X = 0.4*resource_util_C_X + 0.3*latency_X + 0.3*response_time_C
AAI_C_Y = 0.4*resource_util_C_Y + 0.3*latency_Y + 0.3*response_time_C
AAI_C_Z = 0.4*resource_util_C_Z + 0.3*latency_Z + 0.3*response_time_C

print(f'AAIs for App A : {AAI_A_X=}, {AAI_A_Y=}, {AAI_A_Z=}') #<---- which one is minimum? = AAI_A_Z
print(f'App A should be placed on Server Z since {AAI_A_Z=} is minimum\n')

print(f'AAIs for App B : {AAI_B_X=}, {AAI_B_Y=}, {AAI_B_Z=}') #<---- which one is minimum? = AAI_A_Z
print(f'App B should be placed on Server Z since {AAI_B_Z=} is minimum\n')

print(f'AAIs for App C : {AAI_C_X=}, {AAI_C_Y=}, {AAI_C_Z=}') #<---- which one is minimum? = AAI_A_Z
print(f'App C should be placed on Server Z since {AAI_C_Z=} is minimum\n')

print(
"""
Note that server Z does not actually have enough cpu-cores and memory to support all the apps
but that should not be considered in AAI index calculation and application placement decision.
"""
)


AAIs for App A : AAI_A_X=27.4, AAI_A_Y=30.4, AAI_A_Z=24.4
App A should be placed on Server Z since AAI_A_Z=24.4 is minimum

AAIs for App B : AAI_B_X=21.2, AAI_B_Y=24.2, AAI_B_Z=18.2
App B should be placed on Server Z since AAI_B_Z=18.2 is minimum

AAIs for App C : AAI_C_X=33.6, AAI_C_Y=36.6, AAI_C_Z=30.6
App C should be placed on Server Z since AAI_C_Z=30.6 is minimum


Note that server Z does not actually have enough cpu-cores and memory to support all the apps
but that should not be considered in AAI index calculation and application placement decision.



# [Q.7]

![Alt text](image-2.png)

## Solution [7]


![Alt text](image-3.png)

# [Q.8] :: for full solution refer to lecture slides LEC_12_PDQN_DDPG.pptx on slide 24

# [Q.9] [a] Attention calculation

In [2]:
import torch as tt


# define keys
K = tt.tensor([
     [ -0.1458, -0.9395, -0.0372,  0.3079 ], # k1
     [  0.4547, -0.6297, -0.3239, -0.5402 ], # k2
     [  0.0615, -0.1083, -0.1887,  0.9741 ], # k3
]) # size = 3 x 4

# define values
V = tt.tensor([
    [  0.1466, -1.0041 ],  # v1
    [ -0.7882, -0.8074 ],  # v2
    [ -0.2957, -0.1462 ],  # v3
]) # size = 3 x 2

# define queries
Q = tt.tensor([
     [  0.2429,  0.2889,  0.3933, -0.8384 ], # q1
     [  0.1826,  0.3651,  0.5477,  0.7303 ], # q2
]) # size = 2 x 4

print(K.shape, Q.shape, V.shape)


torch.Size([3, 4]) torch.Size([2, 4]) torch.Size([3, 2])


In [3]:
# 1. matrix multiply Q and K

QKt = tt.matmul(Q, K.T)
print(f'Q.Kt is {QKt.shape}\n{QKt}')

Q.Kt is torch.Size([2, 3])
tensor([[-0.5796,  0.2540, -0.9073],
        [-0.1651, -0.7188,  0.5797]])


In [4]:
# 2. Scale by 0.5 (since dimension of keys is 4)
dk = 4
Qkt_scaled = QKt/(dk**0.5)
print(f'Qkt_scaled is {Qkt_scaled.shape}\n{Qkt_scaled}')

Qkt_scaled is torch.Size([2, 3])
tensor([[-0.2898,  0.1270, -0.4536],
        [-0.0826, -0.3594,  0.2899]])


In [5]:
# 3. apply softmax to get weights
attention_weights = tt.softmax(Qkt_scaled, dim=-1)
print(f'attention_weights is {attention_weights.shape}\n{attention_weights}')
# one row for each query

attention_weights is torch.Size([2, 3])
tensor([[0.2971, 0.4507, 0.2522],
        [0.3116, 0.2362, 0.4522]])


In [6]:
# 4. matrix multiply with values

attention_output = tt.matmul(attention_weights, V)
print(f'attention_output is {attention_output.shape}\n{attention_output}')


attention_output is torch.Size([2, 2])
tensor([[-0.3863, -0.6991],
        [-0.2742, -0.5697]])


## for part [b] refer to previously provided solution